# <center>spherical_demo.ipynb
<h1> <center>WESmith 06/29/22

In [1]:
import operator_utils as ws
import sympy as sy

# <center>3D spherical coordinates

In [3]:
ws.test_markdown()

<h3><center> Hello </center></h3>

In [ ]:
aa = ws.coord_xforms()
dd = ws.Operators(aa['spherical_3d'])

In [ ]:
title = 'Transform used in {} coordinates:'.format(dd.name)
ws.latex_block(dd.transform, title=title, center=True, lhs=dd.coords, h_title=1, h_eq=3)

In [ ]:
ws.latex_block(dd.transform, center=False, title=title, h_title=1, lhs=dd.coords, h_eq=2)

In [ ]:
ws.latex_block(dd.Jacobian, title='Jacobian', lhs='J')

In [ ]:
dd.Jacobian_inv

In [ ]:
ws.simp(dd.Jacobian_inv) # simp() reduces expressions element-by-element

In [ ]:
ws.latex_block(ws.simp(dd.Jacobian_inv), title="Jacobian inverse", lhs='inv(J)')

In [ ]:
title = 'Primed basis in {} coordinates:'.format(dd.name)
ws.latex_block(dd.primed_basis, title=title, lhs=dd.Zp, center=False, h_title=1, h_eq=5)

In [ ]:
dd.metric

In [ ]:
dd.metric_sqrt

In [ ]:
dd.metric_inv_sqrt

In [ ]:
dd.det_sqrt

In [ ]:
dd.gradient()

In [ ]:
dd.gradient(vector=False)

In [ ]:
dd.divergence()

In [ ]:
dd.curl()

In [ ]:
dd.curl(vector=False)

In [ ]:
dd.Laplacian_scalar()  # of a general function

In [ ]:
r, theta, phi = dd.coords_p

In [ ]:
# separable function
f, g, h, F = sy.symbols('f g h F', cls=sy.Function)
f = f(r)
g = g(theta)
h = h(phi)
F = f * g * h

In [ ]:
dd.gradient(F)

In [ ]:
title = 'Laplacian of a separable scalar field in {} coordinates'.format(dd.name)
ws.latex_block(dd.Laplacian_scalar(F), title=title, lhs=dd.Laplacian_symbolic(F), h_title=2, h_eq=5)

In [ ]:
title = 'Laplacian of a vector field in {} coordinates'.format(dd.name)
Vname = dd._vector_field()
ws.latex_block(dd.Laplacian_vector_reduced(Vname), title=title, lhs=dd.Laplacian_symbolic(Vname), h_eq=3)

In [ ]:
# Laplacian of f(r) only
dd.Laplacian_scalar(f)

In [ ]:
dd.Laplacian_scalar(1/f) # this is correct

In [ ]:
dd.divergence(dd.gradient(g*h)) # also the Laplacian: products of functions

In [ ]:
dd.Laplacian_vector_full(dd.gradient(h))

In [ ]:
vv = dd.Laplacian_vector_full()

In [ ]:
vv[0]

In [ ]:
# successfully validated against Arfken p.82
vr = dd.Laplacian_vector_reduced()

In [ ]:
vr[0]

In [ ]:
vr[1]

In [ ]:
vr[2]

In [ ]:
dd.material_derivative(vector=False)

In [ ]:
dd.curl(dd.curl())

In [ ]:
dd.gradient(dd.divergence())

In [ ]:
def Arfken_example_2_4_2(ops):
    # ops is an Operators object defined with standard spherical coords r, theta, phi
    # this was successfully validated against Arfken result p. 83 6/18/22
    r, theta, phi = ops.coords_p
    Aphi = sy.symbols('A_phi', cls=sy.Function, real=True)
    Aphi = Aphi(r, theta)
    A    = sy.Matrix([0, 0, Aphi])
    v1   = ops.curl(ops.curl(A))
    tmp  = []
    for j, k in zip(v1, A):
        tmp.append(j + ops.Laplacian_scalar(k) - ops.Laplacian_symbolic(k))
    reduced = sy.trigsimp(ops.Zp.T * sy.Matrix(tmp))
    full    = sy.trigsimp(ops.Zp.T * v1)
    return full, reduced

In [ ]:
full, reduced = Arfken_example_2_4_2(dd)
full

In [ ]:
reduced

# Basic checks

In [ ]:
dd.Laplacian_scalar(1/r)  # should be 0

In [ ]:
dd.gradient(1/r, vector=False)  # should be -1/r**2 in the r_hat direction

In [ ]:
xx = dd.curl(dd.gradient())  # should be 0 vector

In [ ]:
xx

In [ ]:
#(xx.T * xx).args[2][0]
xx._eval_is_zero_matrix()

In [ ]:
dd.divergence(dd.curl())  # should be 0